Real-Time Student Attention Detection System

In [4]:

# IMPORT REQUIRED LIBRARIES

import cv2
import mediapipe as mp
import numpy as np
import time
import math

import pygame


# MediaPipe Setup

mp_face = mp.solutions.face_mesh
mp_pose = mp.solutions.pose

face_mesh = mp_face.FaceMesh(refine_landmarks=True)
pose = mp_pose.Pose()

cap = cv2.VideoCapture(0)


# Alarm Sound Setup

pygame.mixer.init()
alarm_sound = pygame.mixer.Sound("random-alarm.mp3")
alarm_playing = False



# Utility Functions

def dist(p1, p2):
    return math.dist(p1, p2)

def eye_aspect_ratio(eye):
    A = dist(eye[1], eye[5])
    B = dist(eye[2], eye[4])
    C = dist(eye[0], eye[3])
    return (A + B) / (2.0 * C)


# Thresholds

EYE_AR_THRESH = 0.18
SLEEP_TIME_THRESH = 2.0
HEAD_TURN_THRESH = 0.04
SHOULDER_MOVE_THRESH = 0.03


# Timers

eye_close_start = None


# Main Loop

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    h, w, _ = frame.shape
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    face_result = face_mesh.process(rgb)
    pose_result = pose.process(rgb)

    
    # Reset States
    
    eye_closed = False
    sleepy = False
    distracted = False
    shoulders_moved = False
    student_not_detected = True

    
    # FACE ANALYSIS
    
    if face_result.multi_face_landmarks:
        student_not_detected = False
        lm = face_result.multi_face_landmarks[0].landmark

        # Eye EAR
        left_eye_idx = [33, 160, 158, 133, 153, 144]
        left_eye = [(int(lm[i].x * w), int(lm[i].y * h)) for i in left_eye_idx]
        ear = eye_aspect_ratio(left_eye)

        if ear < EYE_AR_THRESH:
            eye_closed = True
            if eye_close_start is None:
                eye_close_start = time.time()
        else:
            eye_close_start = None

        if eye_close_start and (time.time() - eye_close_start) > SLEEP_TIME_THRESH:
            sleepy = True

        # Head Turn Detection
        nose_x = lm[1].x * w
        left_eye_x = lm[33].x * w
        right_eye_x = lm[263].x * w
        face_center_x = (left_eye_x + right_eye_x) / 2

        yaw_ratio = abs(nose_x - face_center_x) / w
        if yaw_ratio > HEAD_TURN_THRESH:
            distracted = True

    
    # POSTURE ANALYSIS
    
    if pose_result.pose_landmarks:
        lm_pose = pose_result.pose_landmarks.landmark
        left_shoulder = lm_pose[mp_pose.PoseLandmark.LEFT_SHOULDER]
        right_shoulder = lm_pose[mp_pose.PoseLandmark.RIGHT_SHOULDER]

        shoulder_diff = abs(left_shoulder.y - right_shoulder.y)
        if shoulder_diff > SHOULDER_MOVE_THRESH:
            shoulders_moved = True

    
    # FINAL DECISION LOGIC
    
    if student_not_detected:
        status = "Student Not Present"
        color = (0, 0, 255)

        if not alarm_playing:
            alarm_sound.play(-1)
            alarm_playing = True

    elif sleepy:
        status = "You are Sleepy"
        color = (0, 0, 255)

        if not alarm_playing:
            alarm_sound.play(-1)
            alarm_playing = True

    elif distracted:
        status = "Student Distracted"
        color = (0, 165, 255)

        alarm_sound.stop()
        alarm_playing = False

    elif not shoulders_moved and not distracted and not eye_closed:
        status = "Engaged"
        color = (0, 255, 0)

        alarm_sound.stop()
        alarm_playing = False

    else:
        status = "Neutral"
        color = (255, 255, 0)

        alarm_sound.stop()
        alarm_playing = False

    
    # DISPLAY
    
    cv2.putText(frame, f"Status: {status}", (30, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, color, 3)

    cv2.imshow("Student Attention & Engagement Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
